# Wine Score Prediction | Wine Recommender

***

# Notebook 4: Preprocessing & Feature Engineering
**Steps to data preprocessing and feature engineering.**

&nbsp;

**By: Annika Scholl**

**BrainStation Data Science**

This notebook is preprocessing the data and implement feature engineering to prepare the data for future modelling. The wine recommender will be focused on the taste and flavor profil of the wine. Therefore, in this notebook each part will be preprocessed. 


***

## Notebook Contents

1. [**Loading Data & Libraries**](#a1)<br>

2. [**Feature Engineering**](#a2)<br>
    2.1 [Taste Profile](#a2.1)<br>
    2.2 [Food Profile](#a2.2)<br>
    2.3 [Flavor Profile](#a2.3)<br>

3. [**Combine Features**](#a3)
        
4. [**Conclusion**](#a4)<br>

***

# Loading Data and Libraries <a id="a1"></a>

In [31]:
# import libraries
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.io as pio

import ast

In [3]:
# Figure settings matplotlib
plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (8,3)
plt.rcParams["figure.dpi"] = 300

In [4]:
# Figure settings plotly
px.defaults.width = 800
px.defaults.height = 600
px.defaults.template = "plotly_dark"

In [5]:
# Read df
df = pd.read_csv(f'data/data_clean.csv', index_col=0)

In [6]:
# Read flavor df
df_flavor = pd.read_csv(f'data/flavor_df.csv', index_col=0)

In [7]:
# Read food df
df_food = pd.read_csv(f'data/food_df.csv', index_col=0)

***

# Feature Engineering <a id="a2"></a>

## Taste Profile

The taste profile consists of the columns `body`, `taste_intensity`, `taste_tannin`, `taste_sweetness`, `taste_acidity` and `taste_fizziness`. For modelling, I am going to actract those columns from the original dataframe.

In [8]:
taste_profil = df.iloc[:,14:20].copy()

In [9]:
taste_profil.head()

,body,taste_intensity,taste_tannin,taste_sweetness,taste_acidity,taste_fizziness
index,,,,,,
4894821_2021,4.0,3.500000,3.000000,2.000000,3.000000,0.0
4541803_2015,4.0,4.000000,3.000000,1.000000,3.000000,0.0
6247930_2020,4.0,3.593151,2.934429,1.526895,3.225890,0.0
4902052_2018,4.0,3.702697,2.678882,1.731447,3.090789,0.0
5232107_2015,4.0,4.000000,3.000000,1.500000,2.000000,0.0


## Food Profile

The food profile is a boolean dataframe. For modelling, I am going to convert it into a binary dataframe.

In [10]:
df_food.head()

,Junk food,Aperitif,Appetizers,Beef,Blue cheese,Cured Meat,Fruity dessert,Game,Goat cheese,Lamb,...,Mild and soft cheese,Mushrooms,Pasta,Pork,Poultry,Rich fish,Shellfish,Spicy food,Sweet dessert,Veal
index,,,,,,,,,,,,,,,,,,,,,
1101361_1992,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
1186041_1959,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
1099538_1929,False,False,False,True,False,False,False,True,False,True,...,False,False,False,False,True,False,False,False,False,False
79235_1993,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
87560_1999,False,False,False,True,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False


In [11]:
col = df_food.columns
for c in col:
    df_food[c] = df_food[c].replace({True: 1, False: 0})

df_food.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46605 entries, 1101361_1992 to 99974_1997
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Junk food               46605 non-null  int64
 1   Aperitif                46605 non-null  int64
 2   Appetizers              46605 non-null  int64
 3   Beef                    46605 non-null  int64
 4   Blue cheese             46605 non-null  int64
 5   Cured Meat              46605 non-null  int64
 6   Fruity dessert          46605 non-null  int64
 7   Game                    46605 non-null  int64
 8   Goat cheese             46605 non-null  int64
 9   Lamb                    46605 non-null  int64
 10  Lean fish               46605 non-null  int64
 11  Mature and hard cheese  46605 non-null  int64
 12  Mild and soft cheese    46605 non-null  int64
 13  Mushrooms               46605 non-null  int64
 14  Pasta                   46605 non-null  int64
 15  Pork    

In [12]:
# Save food profil
food_profil = df_food.copy()

## Flavor Profile

The flavor dataframe requires additional preprocessing and feature engineering. The data is stored as a string that contains a list of multiple dictionaries. The following code provides an overview of the steps to extract the group flavors and their dummies, as well as the subflavors (primary keywords) for the first row/wine. Each flavor and subflavor includes a count of how many people mentioned it. To improve the comparison of different wines, I am adding a 'weight' feature to the composition analysis. This will be used to create dummies and allow for more accurate comparisons, even when the counts are different. 

In [13]:
df_flavor.head()

,flavor
index,
1101361_1992,"[{'group': 'earth', 'stats': {'count': 54, 'sc..."
1186041_1959,"[{'group': 'red_fruit', 'stats': {'count': 188..."
1099538_1929,"[{'group': 'oak', 'stats': {'count': 345, 'sco..."
79235_1993,"[{'group': 'red_fruit', 'stats': {'count': 85,..."
87560_1999,"[{'group': 'red_fruit', 'stats': {'count': 56,..."


### Example of first row

In [14]:
# Extract group flavors for the first row
group_value = df_flavor['flavor'].values[0]
group_value_literal = ast.literal_eval(group_value)
df_group_value = pd.json_normalize(group_value_literal)
df_group_value

,group,primary_keywords,secondary_keywords,stats.count,stats.score
0,earth,"[{'id': 242, 'name': 'leather', 'count': 21}, ...","[{'id': 422, 'name': 'tobacco', 'count': 11}, ...",54,7080
1,red_fruit,"[{'id': 93, 'name': 'cherry', 'count': 23}, {'...","[{'id': 38, 'name': 'black cherry', 'count': 2...",52,6409
2,oak,"[{'id': 422, 'name': 'tobacco', 'count': 11}, ...","[{'id': 242, 'name': 'leather', 'count': 21}, ...",49,4306
3,spices,"[{'id': 320, 'name': 'pepper', 'count': 16}, {...","[{'id': 434, 'name': 'vanilla', 'count': 3}, {...",33,3446
4,non_oak,"[{'id': 98, 'name': 'chestnut', 'count': 1}, {...","[{'id': 384, 'name': 'smoke', 'count': 13}, {'...",33,615
5,black_fruit,"[{'id': 39, 'name': 'black fruit', 'count': 9}...",NaN,29,3542
6,vegetal,"[{'id': 29, 'name': 'bell pepper', 'count': 10...","[{'id': 381, 'name': 'savory', 'count': 2}, {'...",17,1970
7,floral,"[{'id': 322, 'name': 'perfume', 'count': 4}, {...","[{'id': 381, 'name': 'savory', 'count': 2}]",13,1350
8,microbio,"[{'id': 123, 'name': 'cream', 'count': 2}, {'i...","[{'id': 284, 'name': 'mushroom', 'count': 7}, ...",13,833
9,dried_fruit,"[{'id': 147, 'name': 'dried fruit', 'count': 1...","[{'id': 229, 'name': 'jam', 'count': 1}, {'id'...",4,249


In [15]:
# Create new feature 'weight' based on the stats.count to get percentage
df_group_value['weight'] = df_group_value['stats.count']/df_group_value['stats.count'].sum()
df_group_value

,group,primary_keywords,secondary_keywords,stats.count,stats.score,weight
0,earth,"[{'id': 242, 'name': 'leather', 'count': 21}, ...","[{'id': 422, 'name': 'tobacco', 'count': 11}, ...",54,7080,0.181208
1,red_fruit,"[{'id': 93, 'name': 'cherry', 'count': 23}, {'...","[{'id': 38, 'name': 'black cherry', 'count': 2...",52,6409,0.174497
2,oak,"[{'id': 422, 'name': 'tobacco', 'count': 11}, ...","[{'id': 242, 'name': 'leather', 'count': 21}, ...",49,4306,0.164430
3,spices,"[{'id': 320, 'name': 'pepper', 'count': 16}, {...","[{'id': 434, 'name': 'vanilla', 'count': 3}, {...",33,3446,0.110738
4,non_oak,"[{'id': 98, 'name': 'chestnut', 'count': 1}, {...","[{'id': 384, 'name': 'smoke', 'count': 13}, {'...",33,615,0.110738
5,black_fruit,"[{'id': 39, 'name': 'black fruit', 'count': 9}...",NaN,29,3542,0.097315
6,vegetal,"[{'id': 29, 'name': 'bell pepper', 'count': 10...","[{'id': 381, 'name': 'savory', 'count': 2}, {'...",17,1970,0.057047
7,floral,"[{'id': 322, 'name': 'perfume', 'count': 4}, {...","[{'id': 381, 'name': 'savory', 'count': 2}]",13,1350,0.043624
8,microbio,"[{'id': 123, 'name': 'cream', 'count': 2}, {'i...","[{'id': 284, 'name': 'mushroom', 'count': 7}, ...",13,833,0.043624
9,dried_fruit,"[{'id': 147, 'name': 'dried fruit', 'count': 1...","[{'id': 229, 'name': 'jam', 'count': 1}, {'id'...",4,249,0.013423


In [16]:
# Create group dummies using weights
dummy_group = pd.get_dummies(df_group_value['group'], prefix='group') * df_group_value['weight'].values[:, None]

# Sum across rows to collapse into a single row and set index 
result_df_group_value = pd.DataFrame(dummy_group.sum(axis=0)).T
result_df_group_value['index'] = df_flavor['flavor'].index[0]
result_df_group_value.set_index('index',inplace =True)
result_df_group_value

,group_black_fruit,group_citrus_fruit,group_dried_fruit,group_earth,group_floral,group_microbio,group_non_oak,group_oak,group_red_fruit,group_spices,group_vegetal
index,,,,,,,,,,,
1101361_1992,0.097315,0.003356,0.013423,0.181208,0.043624,0.043624,0.110738,0.16443,0.174497,0.110738,0.057047


In [17]:
# Get subflavors and their weight
g_weight = df_group_value.loc[0]['weight']
subgroup = pd.json_normalize(df_group_value['primary_keywords'].values[0])
subgroup['weight'] = subgroup['count']/subgroup['count'].sum()*g_weight
subgroup.head()

,id,name,count,weight
0,242,leather,21,0.047567
1,384,smoke,13,0.029446
2,156,earthy,13,0.029446
3,284,mushroom,7,0.015856
4,276,minerals,6,0.013591


In [18]:
# Create dummies using weights
dummy_subgroup = pd.get_dummies(subgroup['name'], prefix=df_group_value['group'][0]) * subgroup['weight'].values[:, None]

# Sum across rows to collapse into a single row
result_subgroup = pd.DataFrame(dummy_subgroup.sum(axis=0)).T
result_subgroup['index'] = df_flavor['flavor'].index[0]
result_subgroup.set_index('index',inplace =True)
result_subgroup

,earth_chalk,earth_charcoal,earth_cocoa,earth_earthy,earth_forest floor,earth_game,earth_graphite,earth_iron,earth_leather,earth_minerals,earth_mushroom,earth_salt,earth_smoke,earth_stone,earth_tobacco leaf,earth_truffle,earth_underbrush
index,,,,,,,,,,,,,,,,,
1101361_1992,0.002265,0.002265,0.002265,0.029446,0.00453,0.00453,0.011326,0.002265,0.047567,0.013591,0.015856,0.002265,0.029446,0.002265,0.002265,0.00453,0.00453


### For Loop

To extract the flavor and subflavor dummies for each wine in every row, a for loop will be used to add them to a list.

In [19]:
# Empty list to store df for each wine
flavor_group = []
flavor_subgroup = []

# Create range of wines
values = list(range(0,df_flavor.shape[0]))

# for loop for each row
for v in values:
    
    group_value = df_flavor['flavor'].values[v]
    group_value_literal = ast.literal_eval(group_value)
    df_group_value = pd.json_normalize(group_value_literal)

    if not df_group_value.empty:
        # Create weights based on counts and sum
        df_group_value['weight'] = df_group_value['stats.count']/df_group_value['stats.count'].sum()

        # Create group dummies using weights
        dummy_group = pd.get_dummies(df_group_value['group'], prefix='group') * df_group_value['weight'].values[:, None]
        
        # Sum across rows to collapse into a single row and set index 
        result_df_group_value = pd.DataFrame(dummy_group.sum(axis=0)).T
        result_df_group_value['index'] = df_flavor['flavor'].index[v]
        result_df_group_value.set_index('index',inplace =True)

        flavor_group.append(result_df_group_value)

        # Create range for subgroup dummies 
        groups = list(range(0,df_group_value.shape[0]))

        subgroup_list = []

        # Loop through flavor groups in the row
        for g in groups:
            
            # Get weight of the flavor group
            g_weight = df_group_value.loc[g]['weight']

            try:
                # Normalize subgroup
                subgroup = pd.json_normalize(df_group_value['primary_keywords'].values[g])
                

                if not subgroup.empty:
                    subgroup['weight'] = subgroup['count']/subgroup['count'].sum()*g_weight
                    
                    # Create dummies using weights
                    dummy_subgroup = pd.get_dummies(subgroup['name'], prefix=df_group_value['group'][g]) * subgroup['weight'].values[:, None]

                    # Sum across rows to collapse into a single row
                    result_subgroup = pd.DataFrame(dummy_subgroup.sum(axis=0)).T
                    result_subgroup['index'] = df_flavor['flavor'].index[v]
                    result_subgroup.set_index('index',inplace =True)

                    subgroup_list.append(result_subgroup)


            # To avoid error if the primary keywords are empty
            except NotImplementedError:
                continue

        
        flavor_subgroup.append(pd.concat(subgroup_list, axis=1))


    #print(v)

In [20]:
# Concat group flavor into one df and subgroup flavors into one df
flavor_group_df = pd.concat(flavor_group, axis=0)
flavor_subgroup_df = pd.concat(flavor_subgroup, axis=0)

In [21]:
# Merge group and subgroup flavors
merged_df = pd.concat([flavor_group_df, flavor_subgroup_df], axis=1)

# Fill missing values with 0
merged_df.fillna(0, inplace=True)

merged_df.shape

(45563, 468)

In [22]:
# Check df head
merged_df.head()

,group_black_fruit,group_citrus_fruit,group_dried_fruit,group_earth,group_floral,group_microbio,group_non_oak,group_oak,group_red_fruit,group_spices,...,dried_fruit_tamarind candy,tree_fruit_spiced pear,spices_szechuan peppercorn,tree_fruit_canned peach,vegetal_pea shoot,spices_true cinnamon,citrus_fruit_pomello,spices_matcha powder,tree_fruit_unripe pear,spices_jasmine green tea
index,,,,,,,,,,,,,,,,,,,,,
1101361_1992,0.097315,0.003356,0.013423,0.181208,0.043624,0.043624,0.110738,0.164430,0.174497,0.110738,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1186041_1959,0.114108,0.015560,0.016598,0.190871,0.059129,0.047718,0.093361,0.178423,0.195021,0.076763,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1099538_1929,0.177346,0.001942,0.011003,0.196117,0.011003,0.045307,0.117152,0.223301,0.117799,0.086731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79235_1993,0.090411,0.010959,0.008219,0.180822,0.046575,0.049315,0.087671,0.178082,0.232877,0.090411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87560_1999,0.072289,0.060241,0.048193,0.144578,0.024096,0.020080,0.108434,0.136546,0.224900,0.128514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Save flavor profil
flavor_profil = merged_df.copy()

### Chart

In [24]:
group = pd.DataFrame(merged_df.iloc[:,:13].mean(), columns=['mean']).reset_index()
group_list = list(group['index'].str.replace('group_',''))
subgroup_df = pd.DataFrame(merged_df.iloc[:,13:].mean().sort_values().reset_index())
subgroup_list = []

for g in group_list:
    s_df = pd.DataFrame(subgroup_df['index'].str.split(g, expand=True)[1])
    s_df['weight'] = subgroup_df[0] 
    s_df.dropna(inplace=True)
    s_df['parent'] = g
    s_df.rename(columns={1: 'group'}, inplace = True)
    subgroup_list.append(s_df)

chart = pd.concat(subgroup_list)
chart['group'].replace(to_replace='_', value='', regex=True, inplace = True)
chart.loc[chart['group'] == '', 'group'] = chart.loc[chart['group'] == '', 'parent']
chart['parent'].replace(to_replace='_', value=' ', regex=True, inplace = True)

In [33]:
fig = px.sunburst(chart, path=['parent', 'group'], values='weight', color_discrete_sequence=px.colors.qualitative.Antique,
                branchvalues = 'total', title='Average Flavor Profile of Red Wine')
fig.update_layout(width=700,
    height=700,
    font_size=18)
fig.show()

In [ ]:
#Save Figure
pio.write_image(fig, 'data/fig_flavor.png', scale = 10)

***

# Combine Features <a id="a3"></a>

After converting the taste, food, and flavor columns into features, I will combine them into one dataframe. To ensure they are combined correctly, I will check their shape and use an inner join to form the intersection of all features based on their indexes.

In [34]:
taste_profil.head()

,body,taste_intensity,taste_tannin,taste_sweetness,taste_acidity,taste_fizziness
index,,,,,,
4894821_2021,4.0,3.500000,3.000000,2.000000,3.000000,0.0
4541803_2015,4.0,4.000000,3.000000,1.000000,3.000000,0.0
6247930_2020,4.0,3.593151,2.934429,1.526895,3.225890,0.0
4902052_2018,4.0,3.702697,2.678882,1.731447,3.090789,0.0
5232107_2015,4.0,4.000000,3.000000,1.500000,2.000000,0.0


In [35]:
food_profil.head()

,Junk food,Aperitif,Appetizers,Beef,Blue cheese,Cured Meat,Fruity dessert,Game,Goat cheese,Lamb,...,Mild and soft cheese,Mushrooms,Pasta,Pork,Poultry,Rich fish,Shellfish,Spicy food,Sweet dessert,Veal
index,,,,,,,,,,,,,,,,,,,,,
1101361_1992,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1186041_1959,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
1099538_1929,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
79235_1993,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
87560_1999,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
flavor_profil.head()

,group_black_fruit,group_citrus_fruit,group_dried_fruit,group_earth,group_floral,group_microbio,group_non_oak,group_oak,group_red_fruit,group_spices,...,dried_fruit_tamarind candy,tree_fruit_spiced pear,spices_szechuan peppercorn,tree_fruit_canned peach,vegetal_pea shoot,spices_true cinnamon,citrus_fruit_pomello,spices_matcha powder,tree_fruit_unripe pear,spices_jasmine green tea
index,,,,,,,,,,,,,,,,,,,,,
1101361_1992,0.097315,0.003356,0.013423,0.181208,0.043624,0.043624,0.110738,0.164430,0.174497,0.110738,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1186041_1959,0.114108,0.015560,0.016598,0.190871,0.059129,0.047718,0.093361,0.178423,0.195021,0.076763,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1099538_1929,0.177346,0.001942,0.011003,0.196117,0.011003,0.045307,0.117152,0.223301,0.117799,0.086731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79235_1993,0.090411,0.010959,0.008219,0.180822,0.046575,0.049315,0.087671,0.178082,0.232877,0.090411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87560_1999,0.072289,0.060241,0.048193,0.144578,0.024096,0.020080,0.108434,0.136546,0.224900,0.128514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Shape
print(f'Taste Profil: {taste_profil.shape}')
print(f'Food Profil: {food_profil.shape}')
print(f'Flavor Profil: {flavor_profil.shape}')

Taste Profil: (45608, 6)
Food Profil: (46605, 22)
Flavor Profil: (45563, 468)


- The various features have varying shapes due to further cleaning in 2_EDA.ipynb and missing flavor profiles. Therefore, it is important to use an inner join to merge the dataframes and match them by their indexes. 

In [38]:
# Join taste and food profile
feature_tf = taste_profil.join(flavor_profil, how="inner")
feature_tf.shape

(44642, 474)

In [39]:
# Join flavor profil with rest
feature_final = feature_tf.join(food_profil, how="inner")
feature_final.shape

(44642, 496)

In [40]:
feature_final.head()

,body,taste_intensity,taste_tannin,taste_sweetness,taste_acidity,taste_fizziness,group_black_fruit,group_citrus_fruit,group_dried_fruit,group_earth,...,Mild and soft cheese,Mushrooms,Pasta,Pork,Poultry,Rich fish,Shellfish,Spicy food,Sweet dessert,Veal
index,,,,,,,,,,,,,,,,,,,,,
6247930_2020,4.0,3.593151,2.934429,1.526895,3.225890,0.0,0.128205,0.000000,0.025641,0.128205,...,0,0,0,0,0,0,0,0,0,0
4902052_2018,4.0,3.702697,2.678882,1.731447,3.090789,0.0,0.225806,0.000000,0.000000,0.096774,...,0,0,0,0,0,0,0,0,0,0
1202271_2015,4.0,4.727047,3.339554,1.733928,2.881867,0.0,0.098901,0.010989,0.043956,0.032967,...,0,0,0,0,1,0,0,0,0,0
7212283_2022,4.0,4.705246,2.845246,2.044590,2.254098,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,1,0,0,0,0,0,0,0
1551777_2018,4.0,3.862857,2.906767,1.333609,3.282080,0.0,0.183333,0.000000,0.016667,0.133333,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Drop columns with only 1 unique column
unique_counts = feature_final.nunique()
col = unique_counts[unique_counts == 1].index
feature_final.drop(columns=col, inplace = True)

In [42]:
feature_final.head()

,body,taste_intensity,taste_tannin,taste_sweetness,taste_acidity,group_black_fruit,group_citrus_fruit,group_dried_fruit,group_earth,group_floral,...,Lean fish,Mature and hard cheese,Mild and soft cheese,Mushrooms,Pasta,Pork,Poultry,Rich fish,Spicy food,Veal
index,,,,,,,,,,,,,,,,,,,,,
6247930_2020,4.0,3.593151,2.934429,1.526895,3.225890,0.128205,0.000000,0.025641,0.128205,0.025641,...,0,1,0,0,0,0,0,0,0,0
4902052_2018,4.0,3.702697,2.678882,1.731447,3.090789,0.225806,0.000000,0.000000,0.096774,0.000000,...,0,1,0,0,0,0,0,0,0,0
1202271_2015,4.0,4.727047,3.339554,1.733928,2.881867,0.098901,0.010989,0.043956,0.032967,0.000000,...,0,0,0,0,0,0,1,0,0,0
7212283_2022,4.0,4.705246,2.845246,2.044590,2.254098,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,1,0,0,0,0,0
1551777_2018,4.0,3.862857,2.906767,1.333609,3.282080,0.183333,0.000000,0.016667,0.133333,0.016667,...,0,1,0,0,0,0,0,0,0,0


In [43]:
# Save Features to csv
feature_final.to_csv('data/feature_final.csv')

***

# Conclusion <a id="a4"></a>

This notebook prepared the taste, food and flavor features for future modelling.

**Next Steps:**

The following steps involve the modelling including a wine score prediction model that uses linear regression and a recommendation model that includes clustering and cosine similarity.